___
# Supply Planning - Logic

The below is the logic for Net Requirements calculations for finished goods in a company

About the company :
The company has 4 regions of operations which has decicated planning and fulfilment network.
In total including all regions we plan for about 4000SKU's

All data generated are dummy data. The idea of the project is to show how net requirements planning is carried out (MRP)

## Data used as input to the process

> Forecast - Region, SKU level

> Inventory Current - Region, SKU level

> Pruchase Orders - Region, SKU level

> Transfer Orders - Region, SKU level

> MTD sales for consumption - Region, SKU level

> Master data - Categories

> Planning data - Target stockdays, Lead time

## We consider a monthly planning cycle in the below (weekly or daily has similar process)

In [13]:
options("scipen"=100, "digits"=4)
options(scipen = 999)
library(tidyverse)
library(lubridate)
library(readxl)
library(foreach)
library(doParallel)
library(zoo)
options(warn=-1)

> There are two modes to run the report.
> 1. Reorder point calculation : Normal montly ROP for a longer horizon
> 2. Availaibility reporting : This has stringent constraints to check on availability (shorter time period)

In [14]:
duration <- 12
#duration <- 5
plan <- c("Region 1","Region 2","Region 3","Region 4")
report <- "ROP"
#report <- "OOS"

## Data Loading and Cleaning

In [15]:
#Forecast
forecast <- read_xlsx("Data.xlsx", sheet="Forecast") %>% 
  mutate(Date = as.Date(Date)) %>% filter(Forecast.CA > 0)
names(forecast) <- make.names(names(forecast))

In [16]:
head(forecast)

SKU,Date,Plan,Forecast.CA
Product 651,2022-06-01,Region 1,6
Product 651,2022-07-01,Region 1,6
Product 651,2022-09-01,Region 1,7
Product 651,2022-08-01,Region 1,7
Product 651,2022-05-01,Region 1,8
Product 651,2021-07-01,Region 1,20


In [17]:
#Sales
sales <- read_xlsx("Data.xlsx", sheet="Sales") %>%
  mutate(Date = as.Date(Date))
names(sales) <- make.names(names(sales))

In [18]:
head(sales)

SKU,Date,Plan,Sales.CA
Product 651,2021-04-01,Region 1,3
Product 651,2021-06-01,Region 1,6
Product 651,2021-05-01,Region 1,15
Product 651,2021-07-01,Region 1,100
Product 652,2021-04-01,Region 1,134
Product 652,2021-05-01,Region 1,205


In [19]:
#Inventory
inventory <- read_xlsx("Data.xlsx", sheet="Inventory") %>% mutate(Date = Sys.Date())
names(inventory) <- make.names(names(inventory))

In [20]:
head(inventory)

SKU,Plan,Stock.CA,Date
Product 650,Region 1,8.0,2021-07-14
Product 651,Region 1,10.0,2021-07-14
Product 652,Region 1,115.0,2021-07-14
Product 653,Region 1,511.3,2021-07-14
Product 664,Region 1,524.1,2021-07-14
Product 5147,Region 1,20.0,2021-07-14


In [21]:
#Purchase and Transfer
purchase <- read_xlsx("Data.xlsx", sheet="Purchase") %>% mutate(Date = as.Date(Date))
names(purchase) <- make.names(names(purchase))

In [22]:
head(purchase)

SKU,Order.Type,Plan,Date,Purchase.CA
Product 650,OT,Region 1,2021-06-25,0
Product 651,OT,Region 1,2021-07-05,0
Product 651,OT,Region 1,2022-01-10,0
Product 652,OP,Region 1,2021-07-20,601
Product 652,OP,Region 1,2021-09-20,900
Product 652,OT,Region 1,2021-06-25,0


In [23]:
#Master Data
master <- read_xlsx("Data.xlsx", sheet="Master")
names(master) <- make.names(names(master))

New names:
* Desc -> Desc...4
* Desc -> Desc...5


In [24]:
head(master)

Agency,Brand,SKU,Desc...4,Desc...5,Super.Category,Literage,Pack
Supplier 1,Band 1,Product 1,Desc 1,Desc 12,Category 1,4.5,6
Supplier 2,Band 2,Product 2,Desc 2,Desc 13,Category 2,2.1,3
Supplier 2,Band 2,Product 3,Desc 3,Desc 14,Category 2,2.1,3
Supplier 2,Band 2,Product 4,Desc 4,Desc 15,Category 2,6.0,6
Supplier 2,Band 2,Product 5,Desc 5,Desc 16,Category 2,6.0,6
Supplier 2,Band 2,Product 6,Desc 6,Desc 17,Category 2,6.0,6


In [25]:
#Planning
planning <- read_xlsx("Data.xlsx", sheet="Planning")
names(planning) <- make.names(names(planning))

In [26]:
head(planning)

SKU,Master.Planning.Family,Plan,LT,SD
Product 651,PDP,Region 1,85,30
Product 652,PDF,Region 1,85,30
Product 653,PDF,Region 1,85,30
Product 664,PDP,Region 1,85,30
Product 661,PDP,Region 1,70,30
Product 3289,PDF,Region 1,85,30


In [27]:
maxdate <- as.Date(cut(Sys.Date(), "month"))
maxdate

[1] "2021-07-01"

## Cleaning and Aggregation

In [28]:
forecast <- forecast %>% group_by(SKU, Date, Plan) %>% summarise(Forecast.CA = sum(Forecast.CA)) %>% ungroup()

In [29]:
sales <- sales %>% group_by(SKU, Date, Plan) %>% summarise(Sales.CA = sum(Sales.CA)) %>% ungroup()

In [30]:
planning2510 <- planning %>% filter(Plan ==  "Region 2") %>% mutate(Plan = "Region 3")
planning <- bind_rows(planning, planning2510)

In [31]:
purchase <- purchase %>% mutate(Date = if_else(Date < maxdate, maxdate,Date)) %>%
  group_by(SKU, Date, Order.Type, Plan) %>% summarise(Purchase.CA = sum(Purchase.CA)) %>% ungroup() 

In [32]:
transfer <- purchase %>% filter(Order.Type == "OT") %>% mutate(Date = as.Date(cut(Date, "month"))) %>%
  group_by(SKU, Date, Plan) %>% summarise(Transfer.CA = sum(Purchase.CA)) %>% ungroup()

In [33]:
purchase <- purchase %>% filter(Order.Type == "OP") %>% 
  mutate(Date = as.Date(ifelse(day(Date) > 22, ceiling_date(Date,"month"), floor_date(Date,"month")))) %>%
  group_by(SKU, Date, Plan) %>% summarise(Purchase.CA = sum(Purchase.CA)) %>% ungroup()

In [34]:
inventory <- inventory %>% group_by(SKU, Date, Plan) %>% summarise(Stock.CA = sum(Stock.CA)) %>% ungroup() %>%
  mutate(Date = as.Date(cut(today(), "month"))) 

## The daterange and the extractdata function is used to build a planning area

In [35]:
#Build the Grid
startime <- as.Date(cut(today() - 90, "month")) 
datarange <- seq(startime, by = "month", length = 5+duration) 

In [36]:
extractdata <- function(pln, datarange){
  
  forecastx <- forecast %>% filter(Plan == pln) %>% select(-Plan)
  salesx <- sales %>% filter(Plan == pln) %>% select(-Plan)
  inventoryx <- inventory %>% filter(Plan == pln) %>% select(-Plan)
  purchasex <- purchase %>% filter(Plan == pln) %>% select(-Plan)
  planningx <- planning %>% filter(Plan==pln, LT > 0 ) %>% select(-Plan) 

  transferx <- transfer %>% filter(Plan==pln) %>% select(-Plan)
  
  #Step2: Creating a grid with unique set of SKU's and Date Range
  skuactive <- unique(forecastx$SKU)
  data <- expand.grid(SKU = skuactive, Date = datarange) %>%
    arrange(SKU) %>% as.tibble()
  
  data <- data %>% left_join(master) %>% left_join(planningx) %>%
    left_join(salesx) %>% left_join(forecastx) %>% 
    left_join(inventoryx) %>% left_join(purchasex) %>% left_join(transferx)
  error <- data %>% filter(is.na(LT)) %>% mutate(Plan = pln)
  data <- data %>% filter(!is.na(LT))
  
  data <- data %>% mutate(Sales.CA=ifelse(Date <= maxdate & is.na(Sales.CA), 0, Sales.CA)) %>%
    mutate(Stock.CA=ifelse(Date <= maxdate & is.na(Stock.CA), 0, Stock.CA)) %>%
    mutate(Forecast.CA=ifelse(Date <= maxdate & is.na(Forecast.CA), 0, Forecast.CA)) %>%
    mutate(Forecast.CA=ifelse(Date == maxdate, Forecast.CA - Sales.CA, Forecast.CA)) %>%
    mutate(Exs.Consump.CA=ifelse(Date == maxdate & Forecast.CA < 0, -1*Forecast.CA, 0)) %>%
    mutate(Forecast.CA=ifelse(Date == maxdate & Forecast.CA < 0, 0, Forecast.CA)) %>%
    mutate(Forecast.CA=ifelse(Date < maxdate, Sales.CA, Forecast.CA)) %>%
    mutate(Purchase.CA=ifelse(is.na(Purchase.CA), 0, Purchase.CA)) %>%
    mutate(Transfer.CA=ifelse(is.na(Transfer.CA), 0, Transfer.CA))
  
  return(data)
  
}

## Once the planning area is created for each region. The below is a Supply planning logic is used. Here a dynamic calculation on stock levels and safety is done based on days of stock.

![](Capture.PNG)

In [37]:
ROProcessing <- function(data, maxdate, report){
  
  avgforecast <- data %>% arrange(SKU,Date) %>% group_by(SKU) %>%
    mutate(Avg.CA=rollapply(Forecast.CA,3,mean,align='left',fill=NA)) %>% ungroup() %>%
    mutate(Avg.CA=ifelse(is.na(Avg.CA),Forecast.CA,Avg.CA)) %>%
    select(SKU, Date, Avg.CA)

  data <- data %>% mutate(Planned.CA = 0, Closing.inv.CA = 0) %>% rename("Beginning.Inv.CA" = "Stock.CA") %>%
    left_join(avgforecast) %>% 
    mutate(SD.Stocks = SD*Avg.CA/30, StockDays = 0, OOS.CA = 0)
    
  if(report=="OOS"){
    data <- data %>% mutate(Time.Fence = as.Date(ifelse(day(today()+LT) > 20, ceiling_date(today() + LT,"month"), floor_date(today() + LT,"month"))))
  }else{
    data <- data %>% mutate(Time.Fence = as.Date(cut(today()+LT, "month")))
  }

  temp <- data[data$Date >= maxdate,] %>% arrange(SKU, Date)
  data <- data %>% filter(Date < maxdate)
  
  temp <- temp %>% filter(!is.na(Time.Fence))
  temp[is.na(temp)] <- 0
  item <- 0
  
  for (j in 1:length(temp$SKU)){
    
    item <- temp[j,1]
    
    #Keeping maxdate as start date for the ROP for an item. This loop looks into case where ordering is legal
    if (temp[j,]$Date >= temp[j,]$Time.Fence){
      
      if (temp[j,]$Date != maxdate){
        temp[j,]$Beginning.Inv.CA = temp[j-1,]$Closing.inv.CA
      }
      
      if((temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA) < temp[j,]$SD.Stocks){
        temp[j,]$Planned.CA =  temp[j,]$SD.Stocks - (temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA)
      }
      
      #New addition
      if(temp[j,]$Planned.CA + temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA < 0){
        temp[j,]$Closing.inv.CA = 0
        temp[j,]$OOS.CA = -1 * (temp[j,]$Planned.CA + temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA)
      }else{
        temp[j,]$Closing.inv.CA = temp[j,]$Planned.CA + temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA  + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA
      }
      #*****
      
      #The else case looks when the date is less than the time fence and ordering is illegal  
    }else if (temp[j,]$Date < temp[j,]$Time.Fence){
      
      if (temp[j,]$Date != maxdate){
        temp[j,]$Beginning.Inv.CA = temp[j-1,]$Closing.inv.CA
      }
      
      if(temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA < 0){
        temp[j,]$Closing.inv.CA = 0
        temp[j,]$OOS.CA = -1 * (temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA)
      }else{
        temp[j,]$Closing.inv.CA = temp[j,]$Planned.CA + temp[j,]$Beginning.Inv.CA + temp[j,]$Purchase.CA + temp[j,]$Transfer.CA - temp[j,]$Forecast.CA
      }
      
    }else{
      
    }
    
  }

  data <- rbind(data, temp) %>% arrange(SKU, Date)
  data <- data %>% mutate(StockDays = Closing.inv.CA/(Avg.CA/30)) %>%
    mutate(StockDays = ifelse(is.nan(StockDays), 999,StockDays)) %>%
    mutate(StockDays = ifelse(is.infinite(StockDays), 999,StockDays)) %>%
    mutate(Year = format(Date, "%Y"), Month = format(Date, "%b")) %>%
    mutate(Excess.CA = if_else(Date == Time.Fence &  StockDays > SD, Closing.inv.CA-SD.Stocks,0))
    
  temp <- data %>% mutate(Date = as.Date(cut(Date - LT, "month"))) %>%
    mutate(Date = if_else(Date < as.Date(maxdate), as.Date(maxdate), as.Date(Date))) %>%
    rename("Scheduled.CA"="Planned.CA") %>% group_by(SKU, Date) %>%
    summarise(Scheduled.CA = sum(Scheduled.CA)) %>% ungroup()
  
  data <- data %>% left_join(temp) %>% mutate(Scheduled.CA = ifelse(is.na(Scheduled.CA),0,Scheduled.CA))

  return(data)
  
}

## Now we run clanning area creation and supply planning logic on each of the plans

In [38]:
result <- foreach(i=1:length(plan), .combine = rbind.data.frame) %dopar% 
                {
                  data <- extractdata(plan[i], datarange)
                  data <- ROProcessing(data, maxdate, report)
                  cbind.data.frame(Plan=plan[i],data)
                }

Joining, by = "SKU"
Joining, by = "SKU"
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = "SKU"
Joining, by = "SKU"
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = "SKU"
Joining, by = "SKU"
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = "SKU"
Joining, by = "SKU"
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joining, by = c("SKU", "Date")
Joi

## The resultant long format file

In [39]:
head(result)

Plan,SKU,Date,Agency,Brand,Desc...4,Desc...5,Super.Category,Literage,Pack,...,Closing.inv.CA,Avg.CA,SD.Stocks,StockDays,OOS.CA,Time.Fence,Year,Month,Excess.CA,Scheduled.CA
Region 1,Product 1002,2021-04-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.000,0.6667,1.156,0,0,2021-09-01,2021,Apr,0,0.000
Region 1,Product 1002,2021-05-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.000,1.0000,1.733,0,0,2021-09-01,2021,May,0,0.000
Region 1,Product 1002,2021-06-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.000,1.6667,2.889,0,0,2021-09-01,2021,Jun,0,0.000
Region 1,Product 1002,2021-07-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.000,2.3333,4.044,0,3,2021-09-01,2021,Jul,0,11.933
Region 1,Product 1002,2021-08-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.000,2.3333,4.044,0,2,2021-09-01,2021,Aug,0,1.422
Region 1,Product 1002,2021-09-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,4.044,2.3333,4.044,52,0,2021-09-01,2021,Sep,0,7.000


## Lets apply some segmentation data like ABC classification on the above file

In [40]:
abcrun1 <- function(abcvol,x=0.8,y=0.15,z=0.05){
  sc1 <- unique(abcvol$Plan)
  for (j in 1:length(sc1)){
    tempsc <- abcvol[abcvol$Plan == sc1[j],]
    if(j==1){
      abc1 <-abcrun3(tempsc,x,y,z) #ABC Classification
    }else{
      abc1 <- rbind(abc1, abcrun3(tempsc,x,y,z)) #ABC Classification
    }
  }
  return(abc1)
}

abcrun3 <- function(abcvol,x=0.8,y=0.15,z=0.05){
  sc3 <- unique(abcvol$Super.Category)
  for (j in 1:length(sc3)){
    tempsc <- abcvol[abcvol$Super.Category == sc3[j],]
    if(j==1){
      abc3 <-abcbasic(tempsc,x,y,z) #ABC Classification
    }else{
      abc3 <- rbind(abc3, abcbasic(tempsc,x,y,z)) #ABC Classification
    }
  }
  return(abc3)
}


abcbasic <- function(data1, x=0.8,y=0.15,z=0.05){
  
  abc <- data1
  names(abc)[length(names(abc))] <- "values"
  j <- x + y + z
  abc <- arrange(abc, -values)
  cumdecending <- cumsum(arrange(abc, -values)$values)
  cumascending <-cumsum(arrange(abc, values)$values)
  tot <- sum(abc$values)
  cnt <- length(abc$values)
  acutoff <- x * tot
  bcutoff <- y * tot
  ccutoff <- z * tot
  a <- 0
  b <- 0
  c <- 0
  a <- min(which(cumdecending > acutoff))
  if(a < cnt){
    if(ccutoff >= cumascending[1]){
      c <- max(which(cumascending < ccutoff))
    }else if(ccutoff == cumascending[1]){
      c <- 1
    }else{
      c <- 0
    }
  }
  if(a+c < cnt){
    b <- cnt - (a+c)
  }
  if(a+b+c > cnt){
    ABClass <- c(rep("A", a),rep("B", b),rep("C", c-1))
  }else if(a+b+c < cnt){
    ABClass <- c(rep("A", a),rep("B", b),rep("C", c+1))
  }else{
    ABClass <- c(rep("A", a),rep("B", b),rep("C", c))
  }
  abc <- cbind(abc, ABClass)
  for (i in 1:length(names(data1))){
    if(names(data1)[i] == "ItemId"){
      break
    }
  }
  data1 <- merge(x=data1, y = abc[,c(i,length(abc))], by = c("ItemId"), all.x = T)
  return(data1)
  
}


## To apply ABC Classification we load a 6 months sales dataset for the products. At a Region, Category and SKU level

In [41]:
abcvol <- read.csv("abcvol.csv", header=T, sep=",", stringsAsFactors=FALSE)
abc <- abcrun1(abcvol,x=0.8,y=0.15,z=0.05)
write_csv(abc,"abc.csv")

## Combining both datasets

In [42]:
result <- result %>% left_join(abc[,c(1,3,6)], by=c("SKU"="ItemId","Plan"="Plan")) 
result$ABClass[is.na(result$ABClass)] <- "C"

In [43]:
head(result)

Plan,SKU,Date,Agency,Brand,Desc...4,Desc...5,Super.Category,Literage,Pack,...,Avg.CA,SD.Stocks,StockDays,OOS.CA,Time.Fence,Year,Month,Excess.CA,Scheduled.CA,ABClass
Region 1,Product 1002,2021-04-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,0.6667,1.156,0,0,2021-09-01,2021,Apr,0,0.000,C
Region 1,Product 1002,2021-05-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,1.0000,1.733,0,0,2021-09-01,2021,May,0,0.000,C
Region 1,Product 1002,2021-06-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,1.6667,2.889,0,0,2021-09-01,2021,Jun,0,0.000,C
Region 1,Product 1002,2021-07-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,2.3333,4.044,0,3,2021-09-01,2021,Jul,0,11.933,C
Region 1,Product 1002,2021-08-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,2.3333,4.044,0,2,2021-09-01,2021,Aug,0,1.422,C
Region 1,Product 1002,2021-09-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.5,6,...,2.3333,4.044,52,0,2021-09-01,2021,Sep,0,7.000,C


## The results are then downloaded. We will see the summaries for the file in the summary workfile

In [44]:
write.csv(result,"result.csv",row.names = F)

## Now lets Summarize the above results. Please click the below link

[Supply Planning Summary Notebook](SupplyPlanningSummaryandVisualization.ipynb)